# Building Autonomous Agents to Create Analysis Reports -  "Plan and Execute" LangChain agents 

STEPS:
* [1. Setup](#setup)
* [2. Saving Documents on Deep Lake](#saving)
* [3. Creating a Document Retrieval Tool](#retrieval)
* [4. Using the Plan and Execute Agent](#using)


**Plan and Execute agents** are a new type of agent executor offering a different approach than the traditional agents supported in LangChain. These agents are heavily inspired by the BabyAGI framework and the recent Plan-and-Solve paper. Their primary goal is to enable more complex long-term planning, even at the cost of making more calls to the language model.

Each "Plan and Execute" agent consists of:
- **planner component** (utilizes an LLM's reasoning ability to plan out the required steps);
- **executor component** (determines the appropriate tools or actions required to execute each step of the plan).


USEFUL RESOURCES:
- [Plan-and-Execute Agents](https://blog.langchain.dev/plan-and-execute-agents/)
- [Activeloop's Deep Lake](https://python.langchain.com/docs/integrations/vectorstores/activeloop_deeplake)

<hr>
<a class="anchor" id="setup">
    
## 1. Setup
    
</a>

In [1]:
#!pip install langchain==0.0.208 deeplake openai tiktoken

In [2]:
import sys, os
sys.path.append('..')
from keys import OPENAI_API_KEY, ACTIVELOOP_TOKEN

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN

<hr>
<a class="anchor" id="saving">
    
## 2. Saving Documents on Deep Lake
    
</a>

In [3]:
# We scrape several Artificial Intelligence news

import requests
from newspaper import Article # https://github.com/codelucas/newspaper
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/05/23/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/2023/05/18/beijing-launches-campaign-against-ai-generated-misinformation/"
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/11/eu-committees-green-light-ai-act/",
    "https://www.artificialintelligence-news.com/2023/05/09/wozniak-warns-ai-will-power-next-gen-scams/",
    "https://www.artificialintelligence-news.com/2023/05/09/infocepts-ceo-shashank-garg-on-the-da-market-shifts-and-impact-of-ai-on-data-analytics/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/",
    "https://www.artificialintelligence-news.com/2023/04/26/ftc-chairwoman-no-ai-exemption-to-existing-laws/",
    "https://www.artificialintelligence-news.com/2023/04/24/bill-gates-ai-teaching-kids-literacy-within-18-months/",
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/"
]

session = requests.Session()
pages_content = [] # where we save the scraped articles

for url in article_urls:
    try:
        time.sleep(2) # sleep two seconds for gentle scraping
        response = session.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            article = Article(url)
            article.download() # download HTML of webpage
            article.parse() # parse HTML to extract the article text
            pages_content.append({ "url": url, "text": article.text })
        else:
            print(f"Failed to fetch article at {url}")
    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

#If an error occurs while fetching an article, we catch the exception and print
#an error message. This ensures that even if one article fails to download,
#the rest of the articles can still be processed.

In [4]:
# We'll use an embedding model to compute our documents' embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# We'll store the documents and their embeddings in the deep lake vector db
from langchain.vectorstores import DeepLake

# Setup deep lake
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
my_activeloop_org_id = "iryna"
my_activeloop_dataset_name = "langchain_course_analysis_outline"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Your Deep Lake dataset has been successfully created!


In [5]:
# Split the article texts into small chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts = []
for d in pages_content:
    chunks = text_splitter.split_text(d["text"])
    for chunk in chunks:
        all_texts.append(chunk)

In [6]:
len(all_texts)

94

In [7]:
# Add all the chunks to the Deep lake
db.add_texts(all_texts)

Creating embedding data: 100%|███████████████████████████████| 8/8 [00:07<00:00,  1.08it/s]
-

Dataset(path='hub://iryna/langchain_course_analysis_outline', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (94, 1536)  float32   None   
    id        text      (94, 1)      str     None   
 metadata     json      (94, 1)      str     None   
   text       text      (94, 1)      str     None   


['0799cedc-481c-11ee-aa94-12ee7aa5dbdc',
 '0799cfea-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d03a-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d06c-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d09e-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d0d0-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d102-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d134-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d166-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d198-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d1ca-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d1f2-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d224-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d256-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d288-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d2b0-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d2e2-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d314-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d346-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d378-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d3a0-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d3d2-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d404-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d436-481c-11ee-aa94-12ee7aa5dbdc',
 '0799d45e-481c-

<hr>
<a class="anchor" id="retrieval">
    
## 3. Creating a Document Retrieval Tool
    
</a>

In [8]:
# Get the retriever object from the deep lake db object and set the number of retrieved documents to 3
retriever = db.as_retriever()
retriever.search_kwargs['k'] = 3
CUSTOM_TOOL_DOCS_SEPARATOR ="\n---------------\n" # how to join together the retrieved docs to form a single string


# Function that defines the custom tool to retrieve relevant docs from DeepLake
def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = "---------------\n" + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts) + "\n---------------"
    return texts_merged

In [9]:
from langchain.agents.tools import Tool

# Create the tool that uses the "retrieve_n_docs_tool" function
tools = [
    Tool(
        name="Search Private Docs",
        func=retrieve_n_docs_tool,
        description="useful for when you need to answer questions about current events about Artificial Intelligence"
    )
]

<hr>
<a class="anchor" id="using">
    
## 4. Using the Plan and Execute Agent
    
</a>

In [11]:
from langchain.chat_models import ChatOpenAI
#from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

# let's create the Plan and Execute agent
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [14]:
# Test the agent
response = agent.run("Write an overview of Artificial Intelligence regulations in European Union")



> Entering new PlanAndExecute chain...
steps=[Step(value='Research and gather information on the current state of Artificial Intelligence regulations in the European Union.'), Step(value='Identify the key regulations and policies that have been implemented or proposed in the European Union regarding Artificial Intelligence.'), Step(value='Organize the gathered information into different categories such as ethical guidelines, data protection, liability, and transparency.'), Step(value='Summarize the main objectives and principles of the regulations in each category.'), Step(value='Provide examples of specific regulations or guidelines that have been implemented or proposed in the European Union.'), Step(value='Highlight any challenges or controversies surrounding the implementation of these regulations.'), Step(value='Conclude the overview by summarizing the overall impact and significance of Artificial Intelligence regulations in the European Union.'), Step(value="Given the above ste

Thought:Based on the previous steps and the search in the private documents, the key regulations and policies that have been implemented in the European Union regarding Artificial Intelligence (AI) are as follows:

1. AI Act: The EU has recently green-lit the AI Act, which categorizes AI systems based on their potential for harm. The aim of the AI Act is to ensure that AI systems are safe, transparent, traceable, and non-discriminatory.

2. Exemptions for Research Activities and Open-Source AI Components: To boost AI innovation, the AI Act includes exemptions for research activities and AI components provided under open-source licenses.

3. Regulatory Sandboxes: The EU promotes regulatory sandboxes, which are controlled environments established by public authorities, to test AI before its deployment.

4. Citizens' Rights: The AI Act aims to boost citizens' rights by allowing them to file complaints about AI systems and receive explanations of decisions based on high-risk AI systems tha

Action:
```
{
  "action": "Search Private Docs",
  "action_input": {
    "tool_input": "Main objectives and principles of AI regulations in each category"
  }
}
```
Observation: ---------------
The rules are based on a risk-based approach and they establish obligations for providers and users depending on the level of risk that the AI system can generate. AI systems with an unacceptable level of risk to people’s safety would be strictly prohibited, including systems that deploy subliminal or purposefully manipulative techniques, exploit people’s vulnerabilities, or are used for social scoring.
---------------
To boost AI innovation, MEPs added exemptions to these rules for research activities and AI components provided under open-source licenses. The new law also promotes regulatory sandboxes – or controlled environments established by public authorities – to test AI before its deployment.

MEPs want to boost citizens’ right to file complaints about AI systems and receive explanations 

Action:
```
{
  "action": "Search Private Docs",
  "action_input": {
    "tool_input": "Challenges and controversies surrounding the implementation of AI regulations"
  }
}
```
Observation: ---------------
Altman’s call for regulation aligns with concerns raised by AI experts and ethicists, including former Google researchers Dr Timnit Gebru and Meredith Whitaker. They argue that the rapid adoption of AI is overhyped and fails to deliver inherent social good.

Whitaker highlighted the concentration of power in the hands of a few tech companies and their ability to shape social and political landscapes through AI technologies. She expressed concerns about the existing power dynamics and emphasised the need for a more equitable distribution of AI capabilities.

Whitaker cautioned against the notion that AI will automatically lead to social good or equal access for all, arguing that it is a fantasy propagated for marketing purposes. She stressed the importance of acknowledging the concent

Thought:The previous steps have provided a comprehensive overview of the AI regulations in the European Union (EU). The EU has recently green-lit the AI Act, which categorizes AI systems based on their potential for harm. The main objectives of the AI Act are to ensure that AI systems are safe, transparent, traceable, and non-discriminatory. The EU also promotes regulatory sandboxes for testing AI before deployment and has exemptions for research activities and open-source AI components. The EU AI Office is responsible for monitoring the implementation of the AI rulebook.

The impact and significance of AI regulations in the European Union are significant. These regulations aim to protect individuals and ensure that AI systems are developed and used responsibly. By categorizing AI systems based on their potential for harm, the EU is taking a proactive approach to mitigate risks associated with AI. The exemptions for research activities and open-source AI components encourage innovation

In [15]:
print(response)

Based on the information gathered from the previous steps and the search in the private documents, the key regulations and policies that have been implemented in the European Union regarding Artificial Intelligence (AI) are as follows:

1. AI Act: The EU has recently green-lit the AI Act, which categorizes AI systems based on their potential for harm. The aim of the AI Act is to ensure that AI systems are safe, transparent, traceable, and non-discriminatory.

2. Exemptions for Research Activities and Open-Source AI Components: To boost AI innovation, the AI Act includes exemptions for research activities and AI components provided under open-source licenses.

3. Regulatory Sandboxes: The EU promotes regulatory sandboxes, which are controlled environments established by public authorities, to test AI before its deployment.

4. Citizens' Rights: The AI Act aims to boost citizens' rights by allowing them to file complaints about AI systems and receive explanations of decisions based on hi